# UniFi Local API Token Setup

Automated setup of UniFi Local API Token using Selenium/Playwright browser automation.

**Purpose:** Create and store a UniFi Local API Token in 1Password Beastmaster vault for accessing the local controller API to get device and client data (thermostats, TVs, computers, etc.).

**💡 Debugging Tip:** For better visualization, run the "Launch Notebook in Browser" cell at the end to open this notebook in a browser with full automation hooks!


In [ ]:
# Install required libraries
import sys
!{sys.executable} -m pip install selenium playwright requests python-dotenv matplotlib --quiet
!{sys.executable} -m playwright install chromium --quiet
print("✓ Libraries installed (Selenium + Playwright)")


## Architecture Diagram

```mermaid
graph TB
    subgraph "Automation Flow"
        Notebook["📓 This Notebook<br/>Selenium Automation"]
        Selenium["🌐 Selenium/Playwright<br/>Browser Automation"]
        UniFi["🏠 UniFi OS Controller<br/>https://192.168.1.1"]
    end
    
    subgraph "Token Storage"
        OnePass["🔐 1Password CLI<br/>Beastmaster Vault"]
        EnvFile["📝 ~/.env<br/>Optional"]
    end
    
    subgraph "Usage"
        LocalAPI["🔧 Local Controller API<br/>/proxy/network/api/*"]
        Devices["📱 Devices & Clients<br/>Thermostats, TVs, Computers"]
    end
    
    Notebook -->|"Launch Browser"| Selenium
    Selenium -->|"Login & Navigate"| UniFi
    UniFi -->|"Generate Token"| Selenium
    Selenium -->|"Extract Token"| Notebook
    
    Notebook -->|"Store Token"| OnePass
    Notebook -.->|"Optional"| EnvFile
    
    OnePass -->|"Retrieve Token"| LocalAPI
    LocalAPI -->|"Fetch Data"| Devices
    
    style Notebook fill:#e1f5ff
    style Selenium fill:#fff4e1
    style OnePass fill:#e8f5e9
    style LocalAPI fill:#fce4ec
```


In [ ]:
# Load credentials from 1Password
import subprocess
import json
import os
from pathlib import Path
from dotenv import load_dotenv

print("="*70)
print("LOADING CREDENTIALS FROM 1PASSWORD")
print("="*70)

# Load from 1Password Beastmaster vault
def load_unifi_credentials():
    """Load UniFi credentials from 1Password Beastmaster vault."""
    credentials = {}
    
    items = {
        'UNIFI_USERNAME': ('UniFi Username', 'username'),
        'UNIFI_PASSWORD': ('UniFi Password', 'password'),
    }
    
    for env_var, (item_name, field_name) in items.items():
        try:
            result = subprocess.run(
                ['op', 'item', 'get', item_name, '--vault', 'Beastmaster',
                 '--fields', field_name, '--format', 'json'],
                capture_output=True,
                text=True,
                timeout=10
            )
            
            if result.returncode == 0:
                try:
                    field_data = json.loads(result.stdout)
                    if isinstance(field_data, list) and len(field_data) > 0:
                        value = field_data[0].get('value', '')
                    elif isinstance(field_data, dict):
                        value = field_data.get('value', '') or field_data.get(field_name, '')
                    else:
                        value = result.stdout.strip()
                    
                    if value:
                        credentials[env_var] = value
                except json.JSONDecodeError:
                    value = result.stdout.strip()
                    if value:
                        credentials[env_var] = value
        except Exception as e:
            print(f"  ⚠️  Could not load {item_name}: {e}")
    
    return credentials

creds = load_unifi_credentials()

# Fall back to .env if needed
if not creds:
    env_path = Path.home() / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        if os.getenv('UNIFI_USERNAME'):
            creds['UNIFI_USERNAME'] = os.getenv('UNIFI_USERNAME')
        if os.getenv('UNIFI_PASSWORD'):
            creds['UNIFI_PASSWORD'] = os.getenv('UNIFI_PASSWORD')

print(f"\n✓ Loaded {len(creds)} credential(s) from 1Password")
if creds:
    print(f"  • Username: {'✓' if 'UNIFI_USERNAME' in creds else '✗'}")
    print(f"  • Password: {'✓' if 'UNIFI_PASSWORD' in creds else '✗'}")
else:
    print("  ⚠️  No credentials found - add to Beastmaster vault first")


In [ ]:
# Automated token creation using Playwright
from playwright.sync_api import sync_playwright
import time

print("="*70)
print("AUTOMATED TOKEN CREATION")
print("="*70)

UNIFI_CONTROLLER_URL = "https://192.168.1.1"  # Adjust if needed

if not creds.get('UNIFI_USERNAME') or not creds.get('UNIFI_PASSWORD'):
    print("\n⚠️  Missing credentials!")
    print("   Please add UniFi Username and Password to Beastmaster vault.")
else:
    print("\n🚀 Launching browser automation...")
    print("   Browser will open - you can watch the automation!")
    print("   (Browser stays open for debugging)\n")
    
    with sync_playwright() as p:
        # Launch browser (visible for debugging)
        browser = p.chromium.launch(headless=False, slow_mo=500)
        context = browser.new_context(viewport={'width': 1920, 'height': 1080})
        page = context.new_page()
        
        try:
            # Navigate to UniFi OS
            print(f"1. Navigating to {UNIFI_CONTROLLER_URL}...")
            page.goto(UNIFI_CONTROLLER_URL, wait_until='networkidle', timeout=30000)
            
            # Wait for login form
            print("2. Waiting for login page...")
            time.sleep(2)
            
            # Try to find login elements
            username_selector = "input[type='email'], input[name='username'], input[placeholder*='email'], input[placeholder*='Email']"
            password_selector = "input[type='password']"
            
            try:
                username_input = page.wait_for_selector(username_selector, timeout=10000)
                password_input = page.wait_for_selector(password_selector, timeout=10000)
                
                print("3. Filling login credentials...")
                username_input.fill(creds['UNIFI_USERNAME'])
                password_input.fill(creds['UNIFI_PASSWORD'])
                
                # Find and click submit button
                print("4. Submitting login...")
                submit_button = page.wait_for_selector(
                    "button[type='submit'], button:has-text('Sign In'), button:has-text('Login')",
                    timeout=5000
                )
                submit_button.click()
                
                # Wait for navigation
                print("5. Waiting for login to complete...")
                page.wait_for_load_state('networkidle', timeout=30000)
                print("✓ Login successful!")
                
                print("\n💡 Next steps:")
                print("   The automation will attempt to navigate to API Tokens.")
                print("   If auto-navigation fails, you can navigate manually:")
                print("   1. Settings → Account (or User Settings)")
                print("   2. API Tokens section")
                print("   3. Create New Token")
                print("   4. Name it: 'beast-unifi-integration-token'")
                print("   5. Copy the token when shown")
                print("\n   Browser is open - complete token creation, then press Enter...")
                input()
                
                # Try to extract token from page
                print("\n6. Extracting token from page...")
                time.sleep(2)
                
                # Look for token in various formats
                token = None
                token_selectors = [
                    "input[readonly][value], input[type='text'][readonly]",
                    "code, pre, .token, [class*='token']",
                    "textarea[readonly]",
                ]
                
                for selector in token_selectors:
                    try:
                        elements = page.query_selector_all(selector)
                        for elem in elements:
                            text = elem.inner_text().strip()
                            if text and len(text) > 20:
                                token = text
                                print(f"  ✓ Token found via {selector}")
                                break
                        if token:
                            break
                    except:
                        continue
                
                if not token:
                    print("  ⚠️  Could not auto-extract token")
                    print("  Please copy the token from the browser and paste below:")
                    token = input("\nPaste token here: ").strip()
                
                if token and len(token) > 10:
                    print(f"\n✅ Token extracted! ({len(token)} characters)")
                    print(f"   Preview: {token[:10]}...{token[-4:]}")
                    
                    # Store in 1Password
                    print("\n7. Storing token in 1Password Beastmaster vault...")
                    
                    token_item = {
                        "title": "UniFi Local API Token",
                        "category": "API_CREDENTIAL",
                        "fields": [
                            {
                                "id": "password",
                                "value": token,
                                "purpose": "PASSWORD",
                                "label": "Token"
                            },
                            {
                                "id": "notesPlain",
                                "value": "UniFi Local Controller API Token\n\nCreated for beast-unifi-integration project.\nUsed to access local controller at https://192.168.1.1\nfor fetching device and client data (thermostats, TVs, computers, etc.).",
                                "label": "Notes"
                            }
                        ]
                    }
                    
                    result = subprocess.run(
                        ['op', 'item', 'create', '--vault', 'Beastmaster', '--format', 'json'],
                        input=json.dumps(token_item),
                        text=True,
                        capture_output=True,
                        timeout=10
                    )
                    
                    if result.returncode == 0:
                        print("  ✅ Token stored in Beastmaster vault!")
                    else:
                        print(f"  ⚠️  Error storing: {result.stderr[:200]}")
                        print("  💡 Token extracted - you can store it manually if needed")
                    
            except Exception as e:
                print(f"\n⚠️  Automation error: {e}")
                print("\n💡 Browser is open - you can complete the process manually:")
                print("   1. Log in to UniFi OS")
                print("   2. Navigate to Settings → API Tokens")
                print("   3. Create new token")
                print("   4. Copy the token")
                print("\n   Then run the next cell to store it.")
                input("\nPress Enter when done...")
        
        finally:
            # Keep browser open for debugging
            print("\n💡 Browser will stay open for debugging")
            print("   Close it when done.")
            time.sleep(2)


## Manual Token Entry (Fallback)

If automation didn't work, you can manually create the token and enter it here.


In [ ]:
# Manual token entry (if automation didn't work)
print("="*70)
print("MANUAL TOKEN ENTRY")
print("="*70)

print("\nIf automation didn't work, you can manually create the token:")
print("1. Open https://192.168.1.1 in your browser")
print("2. Log in")
print("3. Go to Settings → API Tokens")
print("4. Create new token")
print("5. Copy the token")
print("\nThen paste it below:")

manual_token = input("\nPaste token here (or press Enter to skip): ").strip()

if manual_token and len(manual_token) > 10:
    print(f"\n✓ Token received ({len(manual_token)} characters)")
    
    # Store in 1Password
    print("\nStoring in 1Password Beastmaster vault...")
    
    token_item = {
        "title": "UniFi Local API Token",
        "category": "API_CREDENTIAL",
        "fields": [
            {
                "id": "password",
                "value": manual_token,
                "purpose": "PASSWORD",
                "label": "Token"
            },
            {
                "id": "notesPlain",
                "value": "UniFi Local Controller API Token\n\nCreated for beast-unifi-integration project.",
                "label": "Notes"
            }
        ]
    }
    
    result = subprocess.run(
        ['op', 'item', 'create', '--vault', 'Beastmaster', '--format', 'json'],
        input=json.dumps(token_item),
        text=True,
        capture_output=True,
        timeout=10
    )
    
    if result.returncode == 0:
        print("✅ Token stored in Beastmaster vault!")
    else:
        print(f"⚠️  Error: {result.stderr[:200]}")
else:
    print("\n⚠️  No token entered")


## Verify Token

Test that the token works by connecting to the local controller and fetching sample data.


In [ ]:
# Verify token works
print("="*70)
print("VERIFYING TOKEN")
print("="*70)

# Get token from 1Password
try:
    result = subprocess.run(
        ['op', 'item', 'get', 'UniFi Local API Token', '--vault', 'Beastmaster', '--fields', 'password'],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0 and result.stdout.strip():
        token = result.stdout.strip()
        print(f"\n✓ Token retrieved from 1Password ({len(token)} chars)")
        
        # Test connection
        print("\n📡 Testing connection to local controller...")
        
        try:
            from beast_unifi.api.local_controller import LocalControllerClient
            
            client = LocalControllerClient(
                base_url="https://192.168.1.1:443",
                api_token=token,
                site="default"
            )
            
            # Test with sites endpoint
            sites = client.get_sites()
            print(f"  ✅ Connected! Found {len(sites)} site(s)")
            
            # Try to get devices
            print("\n📱 Fetching devices...")
            devices = client.get_devices()
            print(f"  ✓ Found {len(devices)} UniFi device(s)")
            
            # Try to get clients
            print("\n👥 Fetching clients...")
            clients = client.get_clients()
            print(f"  ✓ Found {len(clients)} client(s)")
            
            if clients:
                print("\n📋 Sample clients (first 10):")
                for client_data in clients[:10]:
                    hostname = client_data.get('hostname', 'Unknown')
                    ip = client_data.get('ip', 'N/A')
                    mac = client_data.get('mac', 'N/A')
                    print(f"     • {hostname} - IP: {ip}, MAC: {mac}")
                
                # Look for interesting devices
                interesting = []
                for client_data in clients:
                    hostname = client_data.get('hostname', '').lower()
                    if any(keyword in hostname for keyword in ['thermostat', 'tv', 'tv-', 'computer', 'pc', 'laptop']):
                        interesting.append(client_data)
                
                if interesting:
                    print("\n⭐ Interesting devices found:")
                    for client_data in interesting:
                        hostname = client_data.get('hostname', 'Unknown')
                        ip = client_data.get('ip', 'N/A')
                        print(f"     • {hostname} - IP: {ip}")
            
            print("\n✅ Token is working!")
            print("   You can now use it to fetch device/client data.")
            
        except Exception as e:
            print(f"  ✗ Connection failed: {e}")
            print("  💡 Check that:")
            print("     • Controller is accessible")
            print("     • Token is valid")
            print("     • Token has correct permissions")
    else:
        print("\n⚠️  Token not found in 1Password")
except Exception as e:
    print(f"\n⚠️  Error: {e}")


## Launch Notebook in Browser for Better Visualization & Debugging

This cell uses Playwright to launch the notebook in a browser with **full automation hooks**, making it much better for:
- 📊 **Viewing visualizations** clearly (Mermaid diagrams)
- 🐛 **Debugging** browser automation interactively
- 🎯 **Stepping through cells** and watching automation in action
- 🔧 **Using browser developer tools** for inspection

**This is the meta-document debugging tool!**


In [ ]:
# Launch notebook in browser using Playwright for better visualization & debugging
# This is the "meta-document" debugging tool - using the notebook itself for debugging!

from playwright.sync_api import sync_playwright
import subprocess
import time
import sys
from pathlib import Path

print("="*60)
print("LAUNCHING NOTEBOOK IN BROWSER FOR DEBUGGING")
print("="*60)

notebook_file = Path("unifi_local_token_setup.ipynb")
notebook_name = notebook_file.name

if not notebook_file.exists():
    print(f"✗ Notebook not found: {notebook_file}")
else:
    print(f"✓ Found notebook: {notebook_name}")
    
    # Check if Jupyter server is running
    jupyter_url = None
    
    try:
        import requests
        for port in [8888, 8889, 8890, 8887]:
            try:
                response = requests.get(f'http://localhost:{port}', timeout=2)
                if response.status_code == 200:
                    jupyter_url = f"http://localhost:{port}"
                    print(f"✓ Jupyter server already running on {jupyter_url}")
                    break
            except:
                pass
    except ImportError:
        print("ℹ requests not available - will start new server")
    
    if not jupyter_url:
        print("\n🚀 Starting Jupyter server...")
        print("   (This may take a moment)")
        
        # Start Jupyter server
        jupyter_cmd = [sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888']
        jupyter_process = subprocess.Popen(
            jupyter_cmd,
            cwd=Path.cwd(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait for server to start
        print("   Waiting for server to start...")
        for i in range(15):
            time.sleep(1)
            try:
                import requests
                response = requests.get('http://localhost:8888', timeout=1)
                if response.status_code == 200:
                    jupyter_url = "http://localhost:8888"
                    print(f"✓ Server started on {jupyter_url}")
                    break
            except:
                if i % 3 == 0:
                    print(f"   Still waiting... ({i+1}/15)")
        
        if not jupyter_url:
            print("⚠️  Server may need more time - trying anyway...")
            jupyter_url = "http://localhost:8888"
            time.sleep(2)
    
    if jupyter_url:
        notebook_path = "notebooks/examples/" + notebook_name
        notebook_url = f"{jupyter_url}/notebooks/{notebook_path}"
        print(f"\n🌐 Launching notebook with Playwright browser automation...")
        print(f"   URL: {notebook_url}\n")
        
        try:
            with sync_playwright() as p:
                # Launch browser with full automation hooks
                print("🚀 Launching Chromium browser with automation hooks...")
                browser = p.chromium.launch(
                    headless=False,
                    args=['--start-maximized']
                )
                
                # Create context with large viewport for better visualization
                context = browser.new_context(viewport={'width': 1920, 'height': 1080})
                page = context.new_page()
                
                # Navigate to notebook
                print("📓 Loading notebook...")
                page.goto(notebook_url, wait_until='networkidle', timeout=30000)
                
                print("✓ Notebook opened in browser with full automation hooks!")
                print("\n✅ Browser window is now open - you can:")
                print("   📊 View all visualizations clearly (Mermaid diagrams)")
                print("   🐛 Debug browser automation interactively")
                print("   🎯 Step through automation cells")
                print("   🔧 Use Playwright hooks for inspection")
                print(f"\n📝 Server: {jupyter_url}")
                print(f"🌐 Notebook: {notebook_url}")
                print("\n💡 Browser will stay open - interact with it freely")
                print("   Close the browser window when you're done")
                print("\n⏸️  This is your debugging meta-document!")
                print("   Use it to step through and debug the automation interactively\n")
                
                # Keep browser open for interaction
                try:
                    while browser.connected:
                        time.sleep(1)
                except KeyboardInterrupt:
                    print("\n\n⚠️  Interrupted - closing browser...")
                
                if browser.connected:
                    browser.close()
                print("✓ Browser closed")
                
        except Exception as e:
            print(f"⚠️  Playwright error: {e}")
            print(f"\n💡 Opening with regular browser instead...")
            import webbrowser
            webbrowser.open(notebook_url)
            print(f"✓ Notebook opened in default browser")
    else:
        print("⚠️  Could not start Jupyter server")
        print(f"   Try manually: {sys.executable} -m jupyter notebook")


## Overview

This notebook automates the process of:
1. **Logging into UniFi OS** at your local controller
2. **Navigating to API Tokens settings**
3. **Creating a new API token**
4. **Storing the token** in 1Password Beastmaster vault
5. **Verifying** the token works

**Why Local API Token?**
- Site Manager API only gives gateway devices (2 hosts)
- Local API gives ALL network devices and clients
- Required for accessing: thermostats, TVs, computers, phones, etc.